In [1]:
!pip install pathlib openml pandas pqdm oslo.concurrency


[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from __future__ import annotations
import pickle
import os
# from pqdm.processes import pqdm
from pathlib import Path
from typing import List, Union

import openml
import pandas as pd
from pqdm.threads import pqdm
from tqdm import tqdm

/Users/smukherjee/.pyenv/versions/3.10.14/envs/openml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Benchmarks

In [2]:
suites =openml.study.list_suites(output_format='dataframe', status='all')

In [3]:
suites.head(4)

,id,alias,main_entity_type,name,status,creation_date,creator
14,14,OpenML100,task,"Collaborative, reproducible benchmarking and a...",in_preparation,2019-02-21 18:40:13,1
99,99,OpenML-CC18,task,OpenML-CC18 Curated Classification benchmark,active,2019-02-21 18:47:13,1
218,218,AutoML-Benchmark,task,AutoML Benchmark,in_preparation,2019-05-02 13:35:08,869
219,219,FOREX,task,Forex,in_preparation,2019-06-04 00:45:17,1


In [4]:
suites.iloc[1].id

99

In [5]:
openml.study.get_suite(99).data

[3,
 6,
 11,
 12,
 14,
 15,
 16,
 18,
 22,
 23,
 28,
 29,
 31,
 32,
 37,
 44,
 46,
 50,
 54,
 151,
 182,
 188,
 38,
 307,
 300,
 458,
 469,
 554,
 1049,
 1050,
 1053,
 1063,
 1067,
 1068,
 1590,
 4134,
 1510,
 1489,
 1494,
 1497,
 1501,
 1480,
 1485,
 1486,
 1487,
 1468,
 1475,
 1462,
 1464,
 4534,
 6332,
 1461,
 4538,
 1478,
 23381,
 40499,
 40668,
 40966,
 40982,
 40994,
 40983,
 40975,
 40984,
 40979,
 40996,
 41027,
 23517,
 40923,
 40927,
 40978,
 40670,
 40701]

In [32]:
def get_suite_tasks(suite_id: int) -> List[int]|None:
    return openml.study.get_suite(suite_id).data

In [35]:
tqdm.pandas()
suites["datasets_related_to_benchmark"] = suites["id"].progress_apply(lambda x: get_suite_tasks(x))

100%|██████████| 91/91 [09:40<00:00,  6.38s/it]


In [ ]:
# 

# Datasets

## Get all data

In [37]:
def get_dataset_description(dataset_id, download_data = False) -> openml.datasets.dataset.OpenMLDataset:

    """
    Get the dataset description from OpenML using the dataset id

    Input: dataset_id (int) : The dataset id

    Returns: data (openml.datasets.dataset.OpenMLDataset) : The dataset object from OpenML
    """
    # TODO : Check for objects that do not have qualities being not downloaded properly
    # try:
    
    data = openml.datasets.get_dataset(
        dataset_id=dataset_id,
        download_data=download_data,
        download_qualities=True,
        download_features_meta_data=True,
    )

    return data

In [50]:
# install the package oslo.concurrency to ensure thread safety
def get_all_metadata_from_openml(n_jobs = 10, download_data = False) -> Union[List, List]:
    """
    Description: Gets all the metadata from OpenML for the type of data specified in the config.
    
    This uses parallel threads (pqdm) and so to ensure thread safety, install the package oslo.concurrency.

    Returns: all the data descriptions combined with data ids, data ids, and the raw openml objects in a dataframe.
    """

    save_filename = f"kaggle_all_dataset_metadata.pkl"
    # Check if the file already exists
    if os.path.exists(save_filename):
        print("[INFO] File already exists. Loading from file.")
        with open(save_filename, "rb") as f:
            openml_data_object, data_id, all_objects = pickle.load(f)
        return openml_data_object, data_id, all_objects
    else:
        # the id column name is different for dataset and flow, so we need to handle that
        id_column_name = "did"

        # Gather all OpenML objects of the type of data
        print("[INFO] Getting dataset metadata.")
        all_objects = openml.datasets.list_datasets(output_format="dataframe")
    
        print("[INFO] Checking downloaded files and skipping them.")

        data_id = [
            int(all_objects.iloc[i][id_column_name])
            for i in range(len(all_objects))
        ]

        # Initialize cache before using parallel (following OpenML python API documentation)
        print("[INFO] Initializing cache.")
        get_dataset_description(data_id[0])

        # Get all object metadata using n_jobs parallel threads from openml
        print("[INFO] Getting dataset metadata from OpenML.")
        openml_data_object = pqdm(
            data_id, get_dataset_description, n_jobs=10
        )
        
        # Save the metadata to a file
        print("[INFO] Saving metadata to file.")
        with open(save_filename, "wb") as f:
            pickle.dump((openml_data_object, data_id, all_objects), f)

        return openml_data_object, data_id, all_objects

## Create dataframe with combined attributes

In [51]:
def extract_attribute(attribute, attr_name):
    """
    Description: Extract an attribute from the OpenML object.

    Input: attribute (object) : The OpenML object

    Returns: The attribute value if it exists, else an empty string.
    """
    return getattr(attribute, attr_name, "")

In [52]:
def create_combined_information_df(
    data_id, descriptions, joined_qualities, joined_features
):
    """
    Description: Create a dataframe with the combined information of the OpenML object.

    Input: data_id (int) : The data id, descriptions (list) : The descriptions of the OpenML object, joined_qualities (list) : The joined qualities of the OpenML object, joined_features (list) : The joined features of the OpenML object

    Returns: The dataframe with the combined information of the OpenML object.
    """
    return pd.DataFrame(
        {
            "did": data_id,
            "description": descriptions,
            "qualities": joined_qualities,
            "features": joined_features,
        }
    )

In [53]:
def combine_metadata(all_dataset_metadata, all_data_description_df):
    """
    Description: Combine the descriptions with the metadata table.

    Input: all_dataset_metadata (pd.DataFrame) : The metadata table, 
    all_data_description_df (pd.DataFrame) : The descriptions

    Returns: The combined metadata table.
    """
    # Combine the descriptions with the metadata table
    all_dataset_metadata = pd.merge(
        all_dataset_metadata, all_data_description_df, on="did", how="inner"
    )

    # Create a single column that has a combined string of all the metadata and the description in the form of "column - value, column - value, ... description"

    # all_dataset_metadata["Combined_information"] = all_dataset_metadata.apply(
    #     merge_all_columns_to_string, axis=1
    # )
    return all_dataset_metadata

In [54]:

def join_attributes(attribute, attr_name):
    """
    Description: Join the attributes of the OpenML object.

    Input: attribute (object) : The OpenML object

    Returns: The joined attributes if they exist, else an empty string.
    example: "column - value, column - value, ..."
    """

    return (
        " ".join([f"{k} : {v}," for k, v in getattr(attribute, attr_name, {}).items()])
        if hasattr(attribute, attr_name)
        else ""
    )

In [55]:
def parse_attributes(attribute_str):
    """
    Reverse the join_attributes function
    """
    attributes = {}
    for item in attribute_str.split(','):
        if ':' in item:
            try:
                k, v = item.split(':')
                attributes[k.strip()] = v.strip()
            except ValueError:
                pass
    return attributes

In [56]:
def create_metadata_dataframe(
    openml_data_object, data_id, all_dataset_metadata, use_cache = False
):
    """
    Description: Creates a dataframe with all the metadata, joined columns with all information for the type of data specified in the config. 
    
    Input: openml_data_object (list) : The list of OpenML objects, data_id (list) : The list of data ids, all_dataset_metadata (pd.DataFrame) : The metadata table, config (dict) : The config dictionary

    Returns: The combined metadata dataframe and the metadata table.
    """
    if use_cache == True:
        # If we are not training, we do not need to recreate the cache and can load the metadata from the files. If the files do not exist, raise an exception.
        try:
            with open(f"kaggle_all_dataset_metadata.csv", "r") as f:
                all_data_description_df = pd.read_csv(f)
            return all_data_description_df
        except:
            raise Exception(
                "Metadata files do not exist. Please run without use_cache."
            )
    else:
        descriptions = [
            extract_attribute(attr, "description") for attr in openml_data_object
        ]
        joined_qualities = [
            join_attributes(attr, "qualities") for attr in openml_data_object
        ]
        joined_features = [
            join_attributes(attr, "features") for attr in openml_data_object
        ]

        all_data_description_df = create_combined_information_df(
            data_id, descriptions, joined_qualities, joined_features
        )
        all_dataset_metadata = combine_metadata(
            all_dataset_metadata, all_data_description_df
        )


        # Expand the qualities column into multiple columns
        qualities_expanded = all_dataset_metadata['qualities'].apply(parse_attributes)
        expanded_df = pd.DataFrame(qualities_expanded.tolist())
        expanded_df = pd.concat([all_dataset_metadata, expanded_df], axis=1)

        expanded_df.to_csv(
            f"kaggle_all_dataset_metadata.csv", index=False
        )

        return expanded_df

## Comparison

In [57]:
def compare_dataset_hash(ds1, ds2, hash_fn):
    return hash_fn(ds1.to_string()) == hash_fn(ds2.to_string())

In [58]:
def get_dataset_sum(df, column_list):
    """
    Description: Get the sum of the columns in the dataframe
    """
    # convert all columns to numeric
    # df[column_list] = df[column_list].apply(pd.to_numeric, errors='coerce')
    df.loc[:, column_list] = df.loc[:, column_list].apply(pd.to_numeric, errors='coerce')

    return abs(df[column_list].values.sum())

In [59]:

def compare_by_func(df1, df2,id1, id2, column_list1, column_list2, func):
    """
    Description: Compare two dataframes using a function given two ids and a list of columns to compare
    TODO : For df2, change the id column to whatever is needed, or add an argument
    """
    return func(df1[df1["did"] == id1], column_list1) == func(df2[df2["did"] == id2], column_list2)


# Collate everything
- This takes a bit of time, even with the dataframes downloaded. 

In [60]:
openml_data_object, data_id, all_metadata = get_all_metadata_from_openml(
        n_jobs=10, download_data=False
    )


[INFO] Getting dataset metadata.
[INFO] Checking downloaded files and skipping them.
[INFO] Initializing cache.
[INFO] Getting dataset metadata from OpenML.


QUEUEING TASKS | :   0%|          | 0/5689 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/5689 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/5689 [00:00<?, ?it/s]

[INFO] Saving metadata to file.


In [62]:
# Create the combined metadata dataframe
metadata_df  = create_metadata_dataframe(
    openml_data_object, data_id, all_metadata, use_cache=False
)


## Getting matching benchmarks

In [71]:
suites

,id,alias,main_entity_type,name,status,creation_date,creator,datasets_related_to_benchmark
14,14,OpenML100,task,"Collaborative, reproducible benchmarking and a...",in_preparation,2019-02-21 18:40:13,1,"[3, 6, 11, 12, 14, 15, 16, 18, 20, 21, 22, 23,..."
99,99,OpenML-CC18,task,OpenML-CC18 Curated Classification benchmark,active,2019-02-21 18:47:13,1,"[3, 6, 11, 12, 14, 15, 16, 18, 22, 23, 28, 29,..."
218,218,AutoML-Benchmark,task,AutoML Benchmark,in_preparation,2019-05-02 13:35:08,869,"[3, 12, 31, 54, 1067, 1111, 1590, 1596, 1489, ..."
219,219,FOREX,task,Forex,in_preparation,2019-06-04 00:45:17,1,"[41708, 41709, 41710, 41711, 41712, 41713, 417..."
225,225,OpenML-friendly,task,OpenML100-friendly,active,2019-09-16 19:41:46,1,"[6, 11, 12, 14, 16, 18, 22, 28, 32, 36, 37, 44..."
...,...,...,...,...,...,...,...,...
425,425,aa97ffae29d04d8caed0209e85b1f6f9,task,mlp-G080-multiclass,in_preparation,2024-03-14 23:54:07,41147,"[1044, 1596, 41168, 42803]"
426,426,d6f80ceebb344f6daacae779fbb1b209,task,mlp-G080-multiclass,in_preparation,2024-03-14 23:54:20,41147,"[1044, 1596, 41168, 42803]"
427,427,875a31da0a6343d2b2dfbf5589678a35,task,",m",in_preparation,2024-03-23 20:10:25,41381,[1]
429,429,6aae78f1fb784deb9975b08870ebd710,task,Regression on omics datasets,in_preparation,2024-06-13 11:00:57,42837,"[46132, 46133, 46134, 46138, 46139, 46140, 461..."


In [106]:
# Initialize a dictionary to hold the results
from collections import defaultdict


dataset_to_suite_ids = defaultdict(list)

# Iterate through the DataFrame and populate the dictionary
for _, row in tqdm(suites.iterrows()):
    suite_id = row['id']
    datasets = row['datasets_related_to_benchmark']
    
    try:
        for dataset in datasets:
                dataset_to_suite_ids[dataset].append(suite_id)
    except TypeError:
        pass

# Convert defaultdict to regular dict for better readability
dataset_to_suite_ids = dict(dataset_to_suite_ids)

91it [00:00, 12257.35it/s]


In [124]:
metadata_df.shape

(5689, 126)

In [126]:
len(dataset_to_suite_ids)

1344

In [127]:
dataset_to_suite_ids

{3: [14, 99, 218, 240, 253, 271],
 6: [14, 99, 225, 253, 281, 285, 293],
 11: [14, 99, 225, 236, 253, 268, 274, 283, 284, 293, 379],
 12: [14, 99, 218, 225, 240, 253, 271],
 14: [14, 99, 225, 236, 253, 379, 388, 389, 390, 391],
 15: [14, 99, 253, 268, 274, 282, 283, 284, 285, 293],
 16: [14, 99, 225, 236, 253, 285, 388, 389, 390, 391],
 18: [14, 99, 225, 236, 253, 268, 274, 283, 284],
 20: [14, 258],
 21: [14],
 22: [14, 99, 225, 236, 253, 285, 379, 388],
 23: [14, 99, 236, 253, 268, 270, 271, 274, 283, 284, 293],
 24: [14, 293],
 28: [14, 99, 225, 236, 253, 258, 285, 293],
 29: [14, 99, 253, 258, 274, 283, 284, 285, 379],
 31: [14, 99, 218, 236, 240, 253, 271, 342, 343, 379, 387, 388],
 32: [14, 99, 225, 236, 253, 258, 285, 293],
 36: [14, 225, 236, 274, 283, 284],
 37: [14, 99, 225, 236, 239, 253, 258, 274, 283, 284, 285, 293, 344, 345],
 42: [14, 293],
 44: [14, 99, 225, 236, 253, 293],
 46: [14, 99, 253, 293, 379],
 50: [14, 99, 253, 258, 274, 283, 284, 285, 293, 344, 345],
 54: [1

In [108]:
def get_openml_url_from_task_id(task_id):
    return f"https://www.openml.org/search?type=task&sort=runs&id={task_id}"

In [114]:
def get_openml_url_from_dataset_id(did):
    return f"https://www.openml.org/search?type=data&sort=runs&id={did}"

In [117]:
get_openml_url_from_task_id(297)

'https://www.openml.org/search?type=task&sort=runs&id=297'

In [128]:
get_openml_url_from_dataset_id(22)

'https://www.openml.org/search?type=data&sort=runs&id=22'

In [133]:
metadata_df["linked_tasks"] = dataset_to_suite_ids

In [147]:
metadata_df["has_linked_tasks"] = metadata_df["linked_tasks"].apply(lambda x: len(x) > 0 if not type(x) == float else False)

In [151]:
metadata_df.head(5)

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,...,RandomTreeDepth2Kappa,RandomTreeDepth3AUC,RandomTreeDepth3ErrRate,RandomTreeDepth3Kappa,StdvNominalAttDistinctValues,kNN1NAUC,kNN1NErrRate,kNN1NKappa,linked_tasks,has_linked_tasks
0,2,anneal,1,1,active,ARFF,684.0,7.0,8.0,5.0,...,0.7953250436852635,0.9296999989655875,0.0801781737193764,0.7953250436852635,1.5576059718800395,0.8721948540771287,0.06347438752783964,0.8261102938928316,NaN,False
1,3,kr-vs-kp,1,1,active,ARFF,1669.0,3.0,1527.0,2.0,...,0.8908498653739637,0.9451363376145694,0.05444305381727159,0.8908498653739637,0.1643989873053572,0.9728427352982838,0.06914893617021277,0.8609676573571107,"[240, 370, 371, 380, 381, 384, 385, 386, 412, ...",True
2,4,labor,1,1,active,ARFF,37.0,3.0,20.0,2.0,...,0.3376623376623376,0.7500711237553342,0.2982456140350877,0.3376623376623376,0.5270462766947299,0.7675675675675676,0.21052631578947367,0.5581395348837209,"[240, 293, 308, 309, 310, 311, 403]",True
3,5,arrhythmia,1,1,active,ARFF,245.0,13.0,2.0,13.0,...,0.23296726097846543,0.6217055780491048,0.5066371681415929,0.23296726097846543,1.3341969312340396,0.5985703851111869,0.4646017699115044,0.21277866242038226,"[14, 99, 218, 240, 253, 271]",True
4,6,letter,1,1,active,ARFF,813.0,26.0,734.0,26.0,...,0.816017080292,0.9080139521161272,0.1769,0.816017080292,0.0,0.9731018928391187,0.05785,0.939833878217649,"[397, 398, 399, 400, 401, 402]",True


: 

In [149]:
metadata_df.to_csv("kaggle_all_dataset_metadata_with_linked_tasks.csv", index=False)

# Mushroom

## Getting the metadata

In [22]:
# version 1 is the oldest one on openml
mushroom = metadata_df[(metadata_df['name'] == 'mushroom') & (metadata_df['version'] == 1)]
mushroom.to_csv('mushroom_metadata.csv', index=False)

In [25]:
mushroom

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,...,RandomTreeDepth2AUC,RandomTreeDepth2ErrRate,RandomTreeDepth2Kappa,RandomTreeDepth3AUC,RandomTreeDepth3ErrRate,RandomTreeDepth3Kappa,StdvNominalAttDistinctValues,kNN1NAUC,kNN1NErrRate,kNN1NKappa
19,24,mushroom,1,1,active,ARFF,4208.0,12.0,3916.0,2.0,...,0.999525,0.000492,0.999014,0.999525,0.000492,0.999014,3.180971,1.0,0.0,1.0


## Get the actual CSV

In [61]:
downloaded_data = get_dataset_description(24, download_data=True)
X, y, _, _ = downloaded_data.get_data(dataset_format='dataframe')

In [95]:
X.to_csv('mushroom_data.csv', index=False)

## Compare dataset hash

In [96]:
# Test using python inbuilt hash function, feel free to use your own
compare_dataset_hash(X, X.T, hash)

False

In [33]:
comparision_list = ['NumberOfInstances', 'NumberOfFeatures', 'NumberOfMissingValues']

In [35]:
# Testing for different datasets
compare_by_func(metadata_df, metadata_df, 24, 25, comparision_list, comparision_list, get_dataset_sum)

False